In [3]:
exec('from __future__ import annotations')
from abc import ABC, abstractmethod
from enum import Enum
import unittest
from typing import NewType, List, Optional, Tuple

In [4]:
Move = NewType('Move', int)

In [5]:
class Piece:
    @property
    def opposite(self):
        """
        This will determine whose turn follows.
        """
        raise NotImplementedError('Should be implemented by subclasses')

In [6]:
class Board(ABC):
    """
    The Board class should be an immutable data structure (i.e. should not be modified).
    Every time a move is played, a new Board instance should be generated to accomodate the move.
    Immutability is useful for the minimax algorithm, because when the search branches it will not
    modify the position of a board from which position are still being analyzed.
    """
    
    @property
    @abstractmethod
    def turn(self) -> Piece:
        """
        Because Tic-Tac-Toe and Connect Four have only one kind of piece,
        a piece can be used as a turn indicator (i.e. who should play).
        """
        ...
    
    @abstractmethod
    def move(self, location: Move):
        ...
        
    @property
    @abstractmethod
    def legal_moves(self) -> List[Move]:
        ...
        
    @property
    @abstractmethod
    def is_win(self) -> bool:
        ...
        
    @property
    def is_draw(self) -> bool:
        return (not self.is_win) and (len(self.legal_moves) == 0)
    
    @abstractmethod
    def evaluate(self, player: Piece) -> float:
        ...

# Tic-Tac-Toe

In [5]:
class TTTPiece(Piece, Enum):
    X = "X"
    O = "O"
    E = " "  # empty
    
    @property
    def opposite(self):
        if self == TTTPiece.O:
            return TTTPiece.X
        elif self == TTTPiece.X:
            return TTTPiece.O
        else:
            return TTTPiece.E
        
    def __str__(self) -> str:
        return self.value
    
class TTTBoard(Board):
    def __init__(
        self,
        position: List[TTTPiece] = [TTTPiece.E] * 9,
        turn: TTTPiece = TTTPiece.X
    ) -> None:
        self.position: List[TTTPiece] = position
        self._turn: TTTPiece = turn
    
    @property
    def turn(self) -> Piece:
        return self._turn
    
    def move(self, location: Move) -> Board:
        temp_position: List[TTTPiece] = self.position.copy()
        temp_position[location] = self._turn
        return TTTBoard(temp_position, self._turn.opposite)
    
    @property
    def legal_moves(self) -> List[Move]:
        return [Move(l) for l in range(len(self.position)) if self.position[l] == TTTPiece.E]
    
    @property
    def is_win(self) -> bool:
        return self.position[0] == self.position[1] and self.position[0] == self.position[2] and \
    self.position[0] != TTTPiece.E or self.position[3] == self.position[4] and \
    self.position[3] == self.position[5] and self.position[3] != TTTPiece.E or \
    self.position[6] == self.position[7] and self.position[6] == self.position[8] and \
    self.position[6] != TTTPiece.E or self.position[0] == self.position[3] and \
    self.position[0] == self.position[6] and self.position[0] != TTTPiece.E or \
    self.position[1] == self.position[4] and self.position[1] == self.position[7] and \
    self.position[1] != TTTPiece.E or self.position[2] == self.position[5] and \
    self.position[2] == self.position[8] and self.position[2] != TTTPiece.E or \
    self.position[0] == self.position[4] and self.position[0] == self.position[8] and \
    self.position[0] != TTTPiece.E or self.position[2] == self.position[4] and \
    self.position[2] == self.position[6] and self.position[2] != TTTPiece.E
    
    def evaluate(self, player: Piece) -> float:
        if self.is_win and self.turn == player:
            return -1
        elif self.is_win and self.turn != player:
            return 1
        else:
            return 0
        
    def __repr__(self) -> str:
        return f"""{self.position[0]}|{self.position[1]}|{self.position[2]}
-----
{self.position[3]}|{self.position[4]}|{self.position[5]}
-----
{self.position[6]}|{self.position[7]}|{self.position[8]}"""

In [6]:
def minimax(board: Board, maximizing: bool, original_player: Piece, max_depth: int = 8) -> float:
    """
    Find the best possible outcome for original player.
    """
    # base case - terminal position or maximal depth reached
    if board.is_win or board.is_draw or max_depth == 0:
        return board.evaluate(original_player)
    
    # recursive case - maximize your gain or minimize the opponent's gains
    if maximizing:
        best_eval: float = float("-inf")  # arbitrarily low starting point
        for move in board.legal_moves:
            result: float = minimax(board.move(move), False, original_player, max_depth - 1)
            best_eval = max(result, best_eval)
        return best_eval
    else:  # minimizing
        worst_eval: float = float("inf")  # arbitrarily high starting point
        for move in board.legal_moves:
            result: float = minimax(board.move(move), True, original_player, max_depth - 1)
            worst_eval = min(result, worst_eval)
        return worst_eval

In [7]:
def find_best_move(board: Board, max_depth: int = 8) -> Move:
    """
    Find the best possible move in the current position.
    Looking up max depth ahead.
    """
    best_eval: float = float("-inf")
    best_move: Move = Move(-1)
    for move in board.legal_moves:
        result: float = minimax(board.move(move), False, board.turn, max_depth)
        if result > best_eval:
            best_eval = result
            best_move = move
    return best_move

In [8]:
class TTTMinimaxTestCase(unittest.TestCase):
    def test_easy_position(self):
        """Win in one move"""
        to_win_easy_position: List[TTTPiece] = [
            TTTPiece.X, TTTPiece.O, TTTPiece.X,
            TTTPiece.X, TTTPiece.E, TTTPiece.O,
            TTTPiece.E, TTTPiece.E, TTTPiece.O
        ]
        test_board: TTTBoard = TTTBoard(to_win_easy_position, TTTPiece.X)
        answer: Move = find_best_move(test_board)
        self.assertEqual(answer, 6)
        
    def test_block_position(self):
        """Must block O's win"""
        to_block_position: List[TTTPiece] = [
            TTTPiece.X, TTTPiece.E, TTTPiece.E,
            TTTPiece.E, TTTPiece.E, TTTPiece.O,
            TTTPiece.E, TTTPiece.X, TTTPiece.O
        ]
        test_board: TTTBoard = TTTBoard(to_block_position, TTTPiece.X)
        answer: Move = find_best_move(test_board)
        self.assertEqual(answer, 2)
        
    def test_hard_position(self):
        """Find the best move to win two moves"""
        to_win_hard_position: List[TTTPiece] = [
            TTTPiece.X, TTTPiece.E, TTTPiece.E,
            TTTPiece.E, TTTPiece.E, TTTPiece.O,
            TTTPiece.O, TTTPiece.X, TTTPiece.E
        ]
        test_board: TTTBoard = TTTBoard(to_win_hard_position, TTTPiece.X)
        answer: Move = find_best_move(test_board)
        self.assertEqual(answer, 1)

In [9]:
board = TTTBoard()

def get_player_move() -> Move:
    player_move: Move = Move(-1)
    while player_move not in board.legal_moves:
        play: int = int(input("Enter a legal square (0-8): "))
        player_move = Move(play)
    return player_move

# main game loop
while True:
    human_move: Move = get_player_move()
    board = board.move(human_move)
    if board.is_win:
        print("Human wins")
        break
    elif board.is_draw:
        print("Draw")
        break
    computer_move: Move = find_best_move(board)
    print(f"Computer move is {computer_move}")
    board = board.move(computer_move)
    print(board)
    if board.is_win:
        print("Computer wins")
        break
    elif board.is_draw:
        print("Draw")
        break

Enter a legal square (0-8): 4
Computer move is 0
O| | 
-----
 |X| 
-----
 | | 
Enter a legal square (0-8): 2
Computer move is 6
O| |X
-----
 |X| 
-----
O| | 
Enter a legal square (0-8): 8
Computer move is 3
O| |X
-----
O|X| 
-----
O| |X
Computer wins


# Connect Four

In [7]:
class C4Piece(Piece, Enum):
    B = "B"
    R = "R"
    E = " "  # stands for empty
    
    @property
    def opposite(self):
        if self == C4Piece.B:
            return C4Piece.R
        elif self == C4Piece.R:
            return C4Piece.B
        else:
            return C4Piece.E
        
    def __str__(self) -> str:
        return self.value
    
def generate_segments(
    num_columns: int,
    num_rows: int,
    segment_length: int
) -> List[List[Tuple[int, int]]]:
    segments: List[List[Tuple[int, int]]] = []

    # generate the vertical segments
    for c in range(num_columns):
        for r in range(num_rows - segment_length + 1):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c, r + t))
            segments.append(segment)

    # generate the horizontal segments
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r))
            segments.append(segment)

    # generate the diagonal segments from bottom left to top right
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows - segment_length + 1):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r + t))
            segments.append(segment)

    # generate the diagonal segments from top left to bottom right
    for c in range(num_columns - segment_length + 1):
        for r in range(segment_length - 1, num_rows):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r - t))
            segments.append(segment)

    return segments

In [10]:
class C4Board(Board):
    NUM_ROWS: int = 6
    NUM_COLUMNS: int = 7
    SEGMENT_LENGTH: int = 4
    SEGMENTS: List[List[Tuple[int, int]]] = generate_segments(NUM_COLUMNS, NUM_ROWS, SEGMENT_LENGTH)
        
    class Column:
        def __init__(self) -> None:
            self._container: List[C4Piece] = []

        @property
        def full(self) -> bool:
            return len(self._container) == C4Board.NUM_ROWS

        def push(self, item: C4Piece) -> None:
            if self.full:
                raise OverflowError("Trying to push piece to full column")
            self._container.append(item)

        def __getitem__(self, index: int) -> C4Piece:
            if index > len(self._container) - 1:
                return C4Piece.E
            return self._container[index]

        def __repr__(self) -> str:
            return repr(self._container)

        def copy(self):
            temp: C4Board.Column = C4Board.Column()
            temp._container = self._container.copy()
            return temp
        
    def __init__(
        self,
        position = None,
        turn: C4Piece = C4Piece.B
    ) -> None:
        if position is None:
            self.position = [C4Board.Column() for _ in range(C4Board.NUM_COLUMNS)]
        else:
            self.position = position
        self._turn: C4Piece = turn
            
    @property
    def turn(self) -> Piece:
        return self._turn
    
    def move(self, location: Move) -> Board:
        temp_position: List[C4Board.Column] = self.position.copy()
        for c in range(C4Board.NUM_COLUMNS):
            temp_position[c] = self.position[c].copy()
        temp_position[location].push(self._turn)
        return C4Board(temp_position, self._turn.opposite)
    
    @property
    def legal_moves(self) -> List[Move]:
        return [Move(c) for c in range(C4Board.NUM_COLUMNS) if not self.position[c].full]
    
    def _count_segment(self, segment: List[Tuple[int, int]]) -> Tuple[int, int]:
        black_count: int = 0
        red_count: int = 0
        for column, row in segment:
            if self.position[column][row] == C4Piece.B:
                black_count += 1
            elif self.position[column][row] == C4Piece.R:
                red_count += 1
        return black_count, red_count
    
    @property
    def is_win(self) -> bool:
        for segment in C4Board.SEGMENTS:
            black_count, red_count = self._count_segment(segment)
            if black_count == 4 or red_count == 4:
                return True
        return False
    
    def _evaluate_segment(self, segment: List[Tuple[int, int]], player: Piece) -> float:
        black_count, red_count = self._count_segment(segment)
        if black_count > 0 and red_count > 0:
            return 0  # mixed segments are neutral
        count: int = max(red_count, black_count)
        score: float = 0
        if count == 2:
            score = 1
        elif count == 3:
            score = 1000
        elif count == 4:
            score = 1000000
        color: C4Piece = C4Piece.B
        if red_count > black_count:
            color = C4Piece.R
        if color != player:
            return -score
        else:
            return score
        
    def evaluate(self, player: Piece) -> float:
        total: float = 0
        for segment in C4Board.SEGMENTS:
            total += self._evaluate_segment(segment, player)
        return total
    
    def __repr__(self) -> str:
        display: str = ""
        for r in reversed(range(C4Board.NUM_ROWS)):
            display += "|"
            for c in range(C4Board.NUM_COLUMNS):
                display += f"{self.position[c][r]} |"
            display += "\n"
        return display

In [17]:
board: Board = C4Board()

def get_player_move() -> Move:
    player_move: Move = Move(-1)
    while player_move not in board.legal_moves:
        play: int = int(input("Enter a column (0-6): "))
        player_move: Move = Move(play)
    return player_move

# main game loop
while True:
    human_move: Move = get_player_move()
    board = board.move(human_move)
    if board.is_win:
        print("Human wins")
        break
    elif board.is_draw:
        print("Draw")
        break
    computer_move: Move = find_best_move(board, 3)
    print(f"Computer move is {computer_move}")
    board = board.move(computer_move)
    print(board)
    if board.is_win:
        print("Computer wins")
        break
    elif board.is_draw:
        print("Draw")
        break

Enter a column (0-6): 3
Computer move is 1
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |R |  |B |  |  |  |

Enter a column (0-6): 1
Computer move is 3
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |B |  |R |  |  |  |
|  |R |  |B |  |  |  |

Enter a column (0-6): 1
Computer move is 1
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |R |  |  |  |  |  |
|  |B |  |  |  |  |  |
|  |B |  |R |  |  |  |
|  |R |  |B |  |  |  |

Enter a column (0-6): 0
Computer move is 4
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |R |  |  |  |  |  |
|  |B |  |  |  |  |  |
|  |B |  |R |  |  |  |
|B |R |  |B |R |  |  |

Enter a column (0-6): 0
Computer move is 0
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|  |R |  |  |  |  |  |
|R |B |  |  |  |  |  |
|B |B |  |R |  |  |  |
|B |R |  |B |R |  |  |

Enter a column (0-6): 0
Computer move is 4
|  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |
|

In [8]:
def alphabeta(
    board: Board,
    maximizing: bool,
    original_player: Piece,
    max_depth: int = 8,
    alpha: float = "-inf",
    beta: float = "inf"
) -> float:
    # base case - max depth or terminal position reached
    if board.is_win or board.is_draw or max_depth == 0:
        return board.evaluate(original_player)
    
    # recursive case - maximize your gains or minimize you opponent's gains
    if maximizing:
        for move in board.legal_moves:
            result: float = alphabeta(board, False, original_player, max_depth - 1, alpha, beta)
            alpha = max(result, alpha)
            if beta <= alpha:
                break
        return alpha
    else: # minimizing
        for move in board.legal_moves:
            result: float = alphabeta(board, True, original_player, max_depth - 1, alpha, beta)
            beta = min(result, beta)
            if beta <= alpha:
                break
        return beta

In [13]:
def find_best_move_alpha_beta(board: Board, max_depth: int = 8) -> Move:
    """
    Find the best possible move in the current position with alphabeta pruning.
    Looking up max depth ahead.
    """
    best_eval: float = float("-inf")
    best_move: Move = Move(-1)
    for move in board.legal_moves:
        result: float = alphabeta(board.move(move), False, board.turn, max_depth)
        if result > best_eval:
            best_eval = result
            best_move = move
    return best_move

board: Board = C4Board()

def get_player_move() -> Move:
    player_move: Move = Move(-1)
    while player_move not in board.legal_moves:
        play: int = int(input("Enter a column (0-6): "))
        player_move: Move = Move(play)
    return player_move

# main game loop
while True:
    human_move: Move = get_player_move()
    board = board.move(human_move)
    if board.is_win:
        print("Human wins")
        break
    elif board.is_draw:
        print("Draw")
        break
    computer_move: Move = find_best_move_alpha_beta(board, 3)
    print(f"Computer move is {computer_move}")
    board = board.move(computer_move)
    print(board)
    if board.is_win:
        print("Computer wins")
        break
    elif board.is_draw:
        print("Draw")
        break

Enter a column (0-6): 4


TypeError: '<' not supported between instances of 'str' and 'int'